In [91]:
with open ("/Users/adityasingh/Desktop/Chatbot/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
print ("Total number of character:", len (raw_text))
print (raw_text [:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [92]:
import re 
text="Hello, world.This, is a test."
result=re.split(r'([,.]|\s)',text)
print(result)

['Hello', ',', '', ' ', 'world', '.', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [93]:
result=re.split(r'([,.]|\s)',text)
print(result)

['Hello', ',', '', ' ', 'world', '.', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [94]:
result=[item for item in result if item.strip()]
print(result)
    

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [95]:
text="Hello, world. Is This--, is a test?"
result=re.split(r'([,.:;?_!()\']|--|\s)',text)
result=[item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'This', '--', ',', 'is', 'a', 'test', '?']


In [96]:
preprocessed=re.split(r'([,.:;?_!"()\']|--|\s)',raw_text)
preprocessed=[item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [97]:
print(len(preprocessed))

4690


Converting token into token id


In [98]:
all_words=sorted(set(preprocessed))
vocab_size=len(all_words)
print(vocab_size)


1130


In [99]:
vocab={token:integer for integer,token in enumerate(all_words)}

In [100]:
for i,item in enumerate(vocab.items()):
    print(item)
    if i>=50:
        break;

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


the above part is a encoder we also want a decoder for reverse mapping for later output to convert token id back to text

Tokenizer Class
(encode method+decode method)

In [101]:
class SimpleTokenizerV1:
    def __init__(self,vocab):
        self.str_to_int=vocab
        self.int_to_str={i:s for s,i in vocab.items()}
    #encoder method
    def encode(self,text):
        preprocessed=re.split(r'([,.:;?_!"()\']|--|\s)',text)
        preprocessed=[
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    #decode method
    def decode(self,ids):
        text=" ".join([self.int_to_str[i] for i in ids])
        #Replace spaces before the specified punctuations
        text=re.sub(r'\s+([,.?!"()\'])',r'\1',text)
        return text

In [102]:
tokenizer=SimpleTokenizerV1(vocab)
text="""It's the last he painted, you know,"Mrs. Gisburn said with pardonable pride."""
ids=tokenizer.encode(text)
print(ids)

[56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [103]:
tokenizer.decode(ids)

'It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

now testing by giving the new sample that is not present in the vocabulaory or in the training set

In [106]:
text="Hello, do you like tea?"
print(tokenizer.encode(text))

KeyError: 'Hello'

So we need a huge dataset to doge this error as we need many words in the vocabulary


In [107]:
all_tokens=sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>","<|unk|>"])

vocab={token:integer for integer,token in enumerate(all_tokens)}

In [108]:
len(vocab.items())

1132

In [109]:
for i,item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [113]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}
    
    # Encoder method
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [114]:
tokenizer=SimpleTokenizerV2(vocab)
text1="Hello, do you like tea?"
text2="In the sunlit terraces of the palace."
text=" <|endoftext|> ".join((text1,text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [115]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [116]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

Based on comparison the de-tokenized text above the original input text,we know that the training set does not contain the words "Hello" and "palace"